In [ ]:
# SQL Alchemy
from sqlalchemy import create_engine

## SQLAlchemy ORM is Flexible  

In [ ]:
# It’s possible to query a database using more SQL:
data = engine.execute("SELECT * FROM BaseballPlayer")
data = engine.execute("SELECT * FROM BaseballPlayer WHERE Rank >= 10")

# or more Python:
players = session.query(BaseballPlayer)
for player in players:
    print(player.name_given)

## Pandas Integrates with SQLAlchemy
### Once we connect to our SQL database using SQLAlchemy
### We can query directly using pandas

In [ ]:
# Pandas
import pandas as pd
# SQL Alchemy
from sqlalchemy import create_engine
# Create Engine
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.create()
# Query All Records in the Database
data = pd.read_sql("SELECT * FROM Census_Data", conn)

## Python Classes

In [ ]:
# Define a class
class Dog():

    # Utilize the Python constructor to initialize the object
    def __init__(self, name, color):
        self.name = name
        self.color = color

# Create an instance of a class
dog = Dog('Fido', 'brown')

# Print the object's attributes
print(dog.name)
print(dog.color)

## SQLAlchemy - SQL with classes already defined (slow method)

In [ ]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [ ]:
# Create Surfer and Board classes
# ----------------------------------
class Surfer(Base):
    __tablename__ = 'surfers'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    hometown = Column(String(255))
    wipeouts = Column(Integer)
    rank = Column(Integer)

class Board(Base):
    __tablename__ = 'surfboards'
    id = Column(Integer, primary_key=True)
    surfer_id = Column(Integer)
    board_name = Column(String(255))
    color = Column(String(255))
    length = Column(Integer)

In [ ]:
# Create specific instances of the Surfer and Board classes
# ----------------------------------
# Create a new surfer named "Bruno"
# Create a new board and associate it with a surfer's ID
surfer = Surfer(name='Bruno', hometown='Gold Coast', wipeouts=4, rank=100)
board = Board(surfer_id=1, board_name='The Beast', color='Blue', length=189)

In [ ]:
# Create Database Connection
# ----------------------------------
# Establish Connection to a sqlite database
engine = create_engine("sqlite:///surfer_db.sqlite")
conn = engine.connect()

In [ ]:
# Create both the Surfer and Board tables within the database
Base.metadata.create_all(engine)
# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [ ]:
# To push the objects made and query the server we use a Session object
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects 
# Add "Bruno" to the current session
# Add "Awwwyeah" to the current session
# Commit both objects to the database
session.add(surfer)
session.add(board)
session.commit()

In [ ]:
# Query the Tables
# ----------------------------------
# Perform a simple query of the database
# Query the database and collect all of the surfers in the Surfer table
surfer_list = session.query(Surfer)
for surfer in surfer_list:
    print(surfer.name)
    print(surfer.id)
board_list = session.query(Board)
for board in board_list:
    print(board.board_name)
    print(board.surfer_id)

In [ ]:
# To add another record for each object
surfer = Surfer(name='Jonno', hometown='Perth', wipeouts=0, rank=1)
board = Board(surfer_id=2, board_name='Awww Yeah', color='Red and Black', length=190)
session.add(surfer)
session.add(board)
session.commit()

## Basic Queries with the ORM - SQLAlchemy 

In [ ]:
# find the number of fatal attacks while surfing
fatal_surfing = session.query(Shark).\
    filter_by(fatal_y_n='Y'). \
    filter_by(activity = 'Surfing').count()
print(fatal_surfing)

# find the number of fatal attacks in Mozambique while spearfishing
fatal_surfing = session.query(Shark).\
    filter_by(fatal_y_n='Y').\
    filter(Shark.country == "MOZAMBIQUE").\
    filter(Shark.activity == 'Spearfishing').count()
print(fatal_surfing)

## CRUD - Update, Delete

In [ ]:
# Update two rows of data
update_one = session.query(Garbage).filter(Garbage.id == 1).first()
update_one.collector = "Jacob Deming"
update_two = session.query(Garbage).filter(Garbage.id == 2).first()
update_two.weight = 11.25
# Commit the updates to the database
session.commit()

In [ ]:
# Delete the row with the lowest weight
session.query(Garbage).filter(Garbage.id == 3).delete()
# Commit the delete to the database
session.commit()

## SQLAlchemy ORM - Reflection

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [ ]:
# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///../Resources/dow.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

# Assign the dow class to a variable called `Dow`
Dow = Base.classes.dow

In [ ]:
# Create a session
session = Session(engine)

# Display the row's columns and data in dictionary format
first_row = session.query(Dow).first()
first_row.__dict__

# Use the session to query Dow table and display the first 5 trade volumes
for row in session.query(Dow.stock, Dow.volume).limit(15).all():
    print(row)

#### Reflecting using SQLAlchemy does not provide users
#### with information on what is being stored.

##### The creators of SQLAlchemy understood this.
* They also created an inspector tool

#### Inspector is used to look up tables, columns and data types.

In [ ]:
# Import SQLAlchemy `automap` and other dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

# Create the connection engine
engine = create_engine("sqlite:///../Resources/dow.sqlite")

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('dow')
for column in columns:
    print(column["name"], column["type"])



#### Queries after Inspection (faster)

In [ ]:
engine.execute("SELECT * from emoji LIMIT 10").fetchall()



#### Reflect database and Query

In [ ]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Emoji = Base.classes.emoji

# Start a session to query the database
session = Session(engine)

# Query Emojis for `emoji_char`, `emoji_id`, and `score` and save the query into results
conn = engine.connect()
results = pd.read_sql("SELECT emoji_char, emoji_id, score FROM emoji", conn)
results.head()

## SQLAlchemy Joining Tables Step-by-Step

1. Use inspect(engine).get_table_names() to find table names in the database
2. Use inspect(engine).get_columns(table) to get the column name
3. Create a list of all table columns you wish to keep
4. Use .filter() to describe what columns to join on


In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

engine = create_engine("sqlite:///../Resources/mammal_masses.sqlite", echo=False)

# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()  
## keys will show all tables in the db - 'ea, and 'na' in this example

# Map classes
EA = Base.classes.ea
NA = Base.classes.na

# create a session
session = Session(engine)

#### Filtering Review
#### Filters are the "WHERE" clause for your select statement

In [ ]:
mammals = session.query(NA).filter(NA.genus == 'Antilocapra').all()
for mammal in mammals:
    print(f"Family: {mammal.family}, Genus: {mammal.genus}")

#### Joins
A SQL join combines columns from one or more tables in a relational database.

It creates a set that can be saved as a table or used as it is.

A JOIN is a means for combining columns from one (self-table) or more tables by using values common to each.

In [ ]:
sel = [EA.family, EA.genus, EA.species, NA.family, NA.genus, NA.species]
same_sporder = session.query(*sel).filter(EA.sporder == NA.sporder).limit(10).all()